In [9]:
#Celda para librerías
import sklearn as sk
import sys
print("Python executable:", sys.executable)

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

#XGBoost
from xgboost import XGBClassifier



from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score


import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

app_train_def = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v1.csv')
#app_train_def_2 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v2.csv')
#app_train_def_3 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v3.csv')
#app_train_def_4 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v4.csv')
#app_train_def_5 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v5.csv')

print(app_train_def.info())
print(app_train_def.shape)

Python executable: /home/yeray/miniconda3/envs/tfg-py3.12/bin/python
1.26.4
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 307 entries, TARGET to WALLSMATERIAL_MODE_Wooden
dtypes: bool(127), float64(138), int64(42)
memory usage: 459.5 MB
None
(307511, 307)


In [10]:
X = app_train_def.drop(columns = ['TARGET'])
y = app_train_def['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.columns = X_train.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)

In [15]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 900, 1800, step=50),
        "max_bin": trial.suggest_int("max_bin", 200, 400, step=10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03, log=True),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "gamma": trial.suggest_float("gamma", 0, 3),
        "min_child_weight": trial.suggest_int("min_child_weight", 12, 24),
        "subsample": trial.suggest_float("subsample", 0.6, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 0.8),
        "n_jobs": trial.suggest_int("n_jobs", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "tree_method": trial.suggest_categorical("tree_method", ["auto", "hist"]),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0),
    }
    model = XGBClassifier(
        **params,
        objective="binary:logistic",
        eval_metric="auc",
        use_label_encoder=False,
        random_state=42
    )

    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model,X_train, y_train, scoring='roc_auc', cv=cv)

    return scores.mean()

In [ ]:
#Estudio para maximizar
study = optuna.create_study(direction="maximize")

#Búsqueda
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best trial:", study.best_trial.number)
print("Best value (mean AUC):", study.best_value)
print("Best hyperparams:", study.best_params)

[I 2025-02-24 12:15:53,089] A new study created in memory with name: no-name-045ec8cc-0df3-4492-a9b3-6b447016dad4


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-02-24 12:22:46,844] Trial 0 finished with value: 0.7667543990569159 and parameters: {'n_estimators': 1700, 'max_bin': 310, 'learning_rate': 0.014233723991802005, 'max_depth': 6, 'gamma': 1.6831388020157805, 'min_child_weight': 18, 'subsample': 0.9235146430824874, 'colsample_bytree': 0.7659923179399154, 'n_jobs': 6, 'reg_alpha': 0.0011931474943634476, 'reg_lambda': 0.0023268484461768063, 'tree_method': 'auto', 'scale_pos_weight': 1.438562453128763}. Best is trial 0 with value: 0.7667543990569159.
[I 2025-02-24 12:26:50,676] Trial 1 finished with value: 0.7588290542284706 and parameters: {'n_estimators': 1650, 'max_bin': 200, 'learning_rate': 0.029782126401585046, 'max_depth': 7, 'gamma': 1.7188850168171097, 'min_child_weight': 13, 'subsample': 0.6751028282743687, 'colsample_bytree': 0.7956257244068641, 'n_jobs': 8, 'reg_alpha': 0.0009707754940263806, 'reg_lambda': 3.7516679367827075e-05, 'tree_method': 'auto', 'scale_pos_weight': 1.6409333544763764}. Best is trial 0 with value: 

In [5]:
import pickle
with open("XGBoost_Optuna_Study_v1_space_reducido_mas_hiperpara.pkl", "wb") as f:
    pickle.dump(study, f)
with open("XGBoost_Optuna_Study_v1_space_reducido_mas_hiperpara.pkl", "rb") as f:
    loaded_study = pickle.load(f)

In [8]:
from tabulate import tabulate

trials_df = loaded_study.trials_dataframe()

df_sorted = trials_df.sort_values(by="value", ascending=False)
best_10 = df_sorted.head(10)

print(tabulate(best_10, headers="keys", tablefmt="psql"))

+----+----------+----------+----------------------------+----------------------------+------------------------+---------------------------+----------------+------------------------+--------------------+---------------------------+-----------------------+--------------------+---------------------+---------------------------+--------------------+----------+
|    |   number |    value | datetime_start             | datetime_complete          | duration               |   params_colsample_bytree |   params_gamma |   params_learning_rate |   params_max_depth |   params_min_child_weight |   params_n_estimators |   params_reg_alpha |   params_reg_lambda |   params_scale_pos_weight |   params_subsample | state    |
|----+----------+----------+----------------------------+----------------------------+------------------------+---------------------------+----------------+------------------------+--------------------+---------------------------+-----------------------+--------------------+---------

In [ ]:
Primera prueba:
|    |   number |    value | datetime_start             | datetime_complete          | duration               |   params_colsample_bytree |   params_gamma |   params_learning_rate |   params_max_depth |   params_min_child_weight |   params_n_estimators |   params_reg_alpha |   params_reg_lambda |   params_scale_pos_weight |   params_subsample | state    |
|----+----------+----------+----------------------------+----------------------------+------------------------+---------------------------+----------------+------------------------+--------------------+---------------------------+-----------------------+--------------------+---------------------+---------------------------+--------------------+----------|
| 35 |       35 | 0.766467 | 2025-02-23 19:05:51.648664 | 2025-02-23 19:23:42.992309 | 0 days 00:17:51.343645 |                  0.609149 |        2.79174 |             0.016566   |                  7 |                        14 |                   950 |        2.98383e-07 |         0.000771683 |                   1.7469  |           0.753771 | COMPLETE |
